<a href="https://colab.research.google.com/github/paul66853/ML/blob/main/PM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import sys

In [48]:
import pandas as pd

In [49]:
import numpy as np

In [50]:
from google.colab import drive

In [51]:
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
data = pd.read_csv('./train.csv',encoding='big5')

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 67.4MB/s]
Archive:  data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [52]:
data=data.iloc[:,3:]#iloc透過行索引的 Index 的具體值來取行資料（如data.iloc[:，:]data[data == 'xxx'] = 0）
print(data)
print(data.shape)
data[data=='NR']=0
print(type(data))
raw_data=data.to_numpy()

         0     1     2     3     4     5  ...    18    19    20    21    22    23
0       14    14    14    13    12    12  ...    17    16    15    15    15    15
1      1.8   1.8   1.8   1.8   1.8   1.8  ...   1.8   1.8   1.8   1.8   1.8   1.8
2     0.51  0.41  0.39  0.37  0.35   0.3  ...  0.56  0.45  0.38  0.35  0.36  0.32
3      0.2  0.15  0.13  0.12  0.11  0.06  ...  0.18  0.12   0.1  0.09   0.1  0.08
4      0.9   0.6   0.5   1.7   1.8   1.5  ...   2.1   1.9   1.5   1.6   1.8   1.5
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
4315   1.8   1.8   1.8   1.8   1.8   1.7  ...     2   1.9   1.9   1.9     2     2
4316    46    13    61    44    55    68  ...   100   109   108   114   108   109
4317    36    55    72   327    74    52  ...   121    97   107   118   100   105
4318   1.9   2.4   1.9   2.8   2.3   1.9  ...   1.5     1   1.7   1.5     2     2
4319   0.7   0.8   1.8     1   1.9   1.7  ...   1.1   1.4   1.3   1.6   1.8     2

[4320 rows x 24

In [53]:
month_data={}
for month in range(12):
  sample = np.empty([18,480])#新建NP數組內含18種數據，20天，每天24小時，總共480個小時
  for day in range(20):
    sample[:,day*24:(day+1)*24]=raw_data[18*(20*month+day):18*(20*month+day+1),:]#從原先的轉換成12*18*(20*24)
  month_data[month]=sample

In [54]:
x = np.empty([12 * 471,18 * 9],dtype = float)
y = np.empty([12 * 471,1],dtype = float)
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month * 471+day * 24 + hour,:] = month_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
      y[month * 471+day * 24 + hour,0] = month_data[month][9,day * 24 + hour + 9]
print(x)
print(x.shape)
print(y)
print(y.shape)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
(5652, 162)
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]
(5652, 1)


In [55]:
mean_x = np.mean(x,axis = 0) #每列的平均值，就是把每天的同一時間點的同一數據做平均值
print(mean_x.shape)       #每列的標準差，就是把每天的同一時間點的同一數據做標準差
std_x = np.std(x,axis = 0)
print(std_x.shape)
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j]-mean_x[j] / std_x[j]) #STD計算標準差
x.shape

(162,)
(162,)


(5652, 162)

In [56]:
import math
x_train_set = x[: math.floor(len(x)*0.8), :] #前80%用於訓練
y_train_set = y[: math.floor(len(y)*0.8), :]  
x_validation = x[math.floor(len(x) *0.8): , :]#20%用於驗證 #floor返回不大於x的最大整數math.floor(100.12):100.0
y_validation = y[math.floor(len(y) *0.8): , :]  
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)

[[10.41419369 10.41440028 10.41509607 ...  0.39218489  0.39193166
  -1.10959063]
 [10.41419369 10.41440028  9.41509607 ...  0.39218489 -1.10806834
  -1.30959063]
 [10.41419369  9.41440028  8.41509607 ... -1.10781511 -1.30806834
  -0.80959063]
 ...
 [24.41419369 23.41440028 21.41509607 ...  1.59218489  0.39193166
  -0.30959063]
 [23.41419369 21.41440028 19.41509607 ...  0.39218489 -0.30806834
  -0.30959063]
 [21.41419369 19.41440028 19.41509607 ... -0.30781511 -0.30806834
  -0.80959063]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 1.94141937e+01  1.94144003e+01  1.94150961e+01 ... -3.07815107e-01
  -8.08068341e-01 -5.09590631e-01]
 [ 1.94141937e+01  1.94144003e+01  2.04150961e+01 ... -8.07815107e-01
  -5.08068341e-01  6.90409369e-01]
 [ 1.94141937e+01  2.04144003e+01  2.04150961e+01 ... -5.07815107e-01
   6.91931659e-01 -9.59063148e-03]
 ...
 [ 1.34141937e+01  1.44144003e+01  1.54150961e+01 ... -5.07815107e-01
  -2.08068341e-01 -3.09590631e-01]
 [ 1.44141937e+01  1.54144003e+01  

In [57]:
dim = 18*9+1  #維度，因為常數項，需多加一欄
w = np.zeros([dim,1]) #生成0的數據
x = np.concatenate((np.ones([12*471,1]),x),axis=1).astype(float)  #拼接1和X
learning_rate = 100 #學習率
iter_time = 1000  #學習次數
adagrad = np.zeros([dim, 1])  #生成0的數據
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):#每訓練100次印出一次LOSS值
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad) + eps) 
np.save('weight.npy', w)
w

0:27.071214829194115
100:228.53922806878893
200:53.69841245217715
300:27.81542588798013
400:19.344724695548447
500:15.557117138450378
600:13.509561287195009
700:12.211436460143375
800:11.29276749929352
900:10.595655828529354


array([[-1.24891458e-01],
       [-5.50715736e-04],
       [-2.00485760e-02],
       [-3.18771490e-02],
       [-3.99865095e-02],
       [-4.87519832e-02],
       [-6.54844835e-02],
       [-8.28565321e-02],
       [-1.03417795e-01],
       [-1.30788321e-01],
       [ 1.20245668e-01],
       [ 1.20353906e-01],
       [ 1.20151456e-01],
       [ 1.21121085e-01],
       [ 1.20258089e-01],
       [ 1.19500050e-01],
       [ 1.19188784e-01],
       [ 1.18830530e-01],
       [ 1.19364482e-01],
       [ 1.04286199e-01],
       [ 1.01443269e-01],
       [ 1.14923042e-01],
       [ 1.12709078e-01],
       [ 1.23745027e-01],
       [ 1.56351071e-01],
       [ 1.63485463e-01],
       [ 2.03887232e-01],
       [ 1.26930494e-01],
       [ 1.51638807e-01],
       [ 1.51586913e-01],
       [ 1.48923433e-01],
       [ 1.50410705e-01],
       [ 1.48711746e-01],
       [ 1.57603421e-01],
       [ 1.60255214e-01],
       [ 1.44029110e-01],
       [ 1.21947486e-01],
       [ 4.85839257e-01],
       [ 3.6

In [58]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]  #去除文件第一列的ID和名稱，只留下數據
test_data[test_data == 'NR'] = 0  #跟前面一樣，將NR改成0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)#18*9是特徵的數量，240組，每組18*9個
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)#每隔18個數據為一行
for i in range(len(test_x)):      #行
    for j in range(len(test_x[0])): #列
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [59]:
w = np.load('weight.npy')  # 讀取剛剛的文檔
ans_y = np.dot(test_x, w)  # 預測
ans_y

array([[ 4.32518045e-01],
       [-1.74842430e+00],
       [ 5.75239771e+00],
       [-2.96515082e+00],
       [ 1.60432319e+00],
       [-6.10918491e-01],
       [-1.73835868e+00],
       [ 2.04637600e+00],
       [-1.67234729e+00],
       [ 1.87563052e+01],
       [-1.95087777e+00],
       [-1.67032174e+01],
       [ 2.99909793e+00],
       [ 1.10240380e+01],
       [-1.12847043e+00],
       [-4.76995336e+00],
       [ 3.73514000e+00],
       [ 9.97518202e+00],
       [-9.87418093e+00],
       [-2.58395720e+00],
       [ 5.10527263e+00],
       [ 1.30561577e+01],
       [-2.62075176e+00],
       [-2.01172439e+00],
       [-3.02997435e+00],
       [ 6.55248994e+00],
       [ 2.61497905e+00],
       [ 1.10081313e+01],
       [-1.61308446e+00],
       [ 2.08513495e+01],
       [ 1.30979217e+01],
       [ 2.93691786e+00],
       [-9.08456352e-01],
       [ 3.29550368e+00],
       [ 4.52670129e+00],
       [ 1.28556411e+00],
       [ 3.82310484e+00],
       [ 6.15944208e+00],
       [ 1.8

In [60]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 0.4325180451249129]
['id_1', -1.7484242988378396]
['id_2', 5.7523977053280655]
['id_3', -2.9651508203741375]
['id_4', 1.6043231937308793]
['id_5', -0.6109184911230597]
['id_6', -1.7383586762219212]
['id_7', 2.046376003169975]
['id_8', -1.672347286968725]
['id_9', 18.756305176307198]
['id_10', -1.950877767457104]
['id_11', -16.70321735864337]
['id_12', 2.999097934402862]
['id_13', 11.024037993890454]
['id_14', -1.1284704256972968]
['id_15', -4.769953364884512]
['id_16', 3.7351399960290745]
['id_17', 9.975182024052089]
['id_18', -9.874180933323647]
['id_19', -2.583957196447641]
['id_20', 5.105272627208619]
['id_21', 13.056157681496297]
['id_22', -2.6207517568293737]
['id_23', -2.0117243900977417]
['id_24', -3.02997434513144]
['id_25', 6.552489938849109]
['id_26', 2.614979054836972]
['id_27', 11.008131260124818]
['id_28', -1.6130844560383069]
['id_29', 20.851349508777616]
['id_30', 13.097921664605682]
['id_31', 2.9369178566544702]
['id_32', -0.9084563521431669]
['